The idea of the visualization and the base code was taken from https://github.com/AIML-IfI/openset-imagenet-comparison. It was developed **by Palechor, Andres and Bhoumik, Annesha and Günther, Manuel** for the paper "**Large-Scale Open-Set Classification Protocols for ImageNet**" (Winter Conference on Applications of Computer Vision (WACV)) in 2023. You can find the citation in the readme file.

Adaptions have been made to visualize a different version of the table-layout.  

In [ ]:
import argparse
import multiprocessing
import collections
import subprocess
import pathlib
import os, sys
import torch
import numpy

from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

from collections import defaultdict

from matplotlib import pyplot, cm, colors
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.ticker import MaxNLocator, LogLocator

from pathlib import Path
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib import ticker
from matplotlib.lines import Line2D
from matplotlib.ticker import LogLocator, NullFormatter
from matplotlib import colors
import matplotlib.cm as cm

import yaml

THRESHOLDS = {1e-4: "$10^{-4}$",
              1e-3: "$10^{-3}$",
              1e-2: "$10^{-2}$",
              1e-1: "$10^{-1}$",
              0.2: "$0.2$",
              1: "$1$",
}



# get distinguishable colors
import matplotlib.cm
colors = matplotlib.cm.tab10(range(10))

COLORS = {
    "threshold": colors[0],
    "openmax": colors[8],
    "proser": colors[2],
    "evm": colors[3],
    "maxlogits": colors[5]
}

STYLES = {
    "entropic": "dashed",
    "softmax": "solid",
    "garbage": "dotted",
    "p1": "dashed",
    "p2": "dotted",
    "p3": "solid"
}

NAMES = {
    "threshold": "Threshold",
    "openmax": "OpenMax",
    "proser": "PROSER",
    "evm": "EVM",
    "maxlogits": "MaxLogits",
    "entropic": "EOS",
    "softmax": "Softmax",
    "garbage": "Garbage",
    "p1": "P_1",
    "p2": "P_2",
    "p3": "P_3",
    1: "$P_1$",
    2: "$P_2$",
    3: "$P_3$"
}

In [ ]:
#function to load the scores and ground truths
def load_scores(use_best, algorithm, loss, protocol ,method, eval, experiment_number):
    suffix = "best" if use_best else "curr"
    scores = defaultdict(lambda: defaultdict(dict))
    ground_truths = {}
    scr = "scores"

    score_file = f"{loss}_{algorithm}_{eval}_arr_{suffix}_{str(experiment_number)}.npz"

    if(os.path.exists(score_file)):
        results = numpy.load(score_file)
        scores[protocol][loss][algorithm] = results[scr]
        if(protocol not in ground_truths):
            ground_truths[protocol] = results["gt"].astype(int)
        else:
            assert numpy.all(results["gt"] == ground_truths[protocol])

    else:
      print("did not find the data")

    return scores, ground_truths

#function to calculate the CCR at FPR-values
def ccr_at_fpr(gt, scores, fpr_values, unk_label=-1):

    # compute ccr and fpr values from scores
    ccr, fpr = calculate_oscr(gt, scores, unk_label)

    ccrs = []
    for t in fpr_values:
        # get the FPR value that is closest, but above the current threshold
        candidates = np.nonzero(np.maximum(t - fpr, 0))[0]
        if candidates.size > 0:
            ccrs.append(ccr[candidates[0]])
        else:
            ccrs.append(None)

    return ccrs

In [ ]:
#function to write a table with the CCR at FPR-values
def ccr_table_new(protocols,protocols2,protocols3,title_column,title_1, title_2, title_3,algorithm, loss, scores, gt, scores2, gt2, scores3, gt3, output_name, caption):
      """
      Input:

        protocols = protocol number of scores, gt
        protocols2 = protocol number of scores2, gt2
        protocols3 = protocol number of scores3, gt3

        output_name = name of the file
        caption = define the caption for the table
        title_column = the title of the first column which defines the different methods
        title_1 to title_3 = title of the tree data inputs (scores,gt), (scores2,gt2),(scores3,gt3)
        algorithm = the used algorithm for the method data
        loss = the used loss function for the method data

      """

      #parameter defined
      fpr_thresholds = [1e-3,1e-2,1e-1, 1.]
      #losses = ('entropic')
      #algorithms = ('threshold')

      def nonemax(a,b):
          b = numpy.array([v if v is not None else numpy.nan for v in b])
          return numpy.where(numpy.logical_and(numpy.logical_not(numpy.isnan(b)), b >= a), b, a)

      latex_file = f"{output_name}.tex"
      print("Writing CCR tables for protocol", protocol, "to file", latex_file)

      # compute all CCR values and store the values for protocol 1
      results_n_p1 = collections.defaultdict(dict)
      max_total_n_p1 = numpy.zeros(len(fpr_thresholds))
      results_u_p1 = collections.defaultdict(dict)
      max_total_u_p1 = numpy.zeros(len(fpr_thresholds))

      ccrs_p1 = ccr_at_fpr(gt[protocols], scores[protocols][loss][algorithm], fpr_thresholds, unk_label=-1)
      results_n_p1[algorithm][loss] = ccrs_p1
      max_total_n_p1 = nonemax(max_total_n_p1, ccrs_p1)

      ccrs_p1 = ccr_at_fpr(gt[protocols], scores[protocols][loss][algorithm], fpr_thresholds, unk_label=-2)
      results_u_p1[algorithm][loss] = ccrs_p1
      max_total_u_p1 = nonemax(max_total_u_p1, ccrs_p1)

      total_u_p1 = max_total_u_p1
      total_u_p1 = np.append(total_u_p1,sum(max_total_u_p1))

      total_n_p1 = max_total_n_p1
      total_n_p1 = np.append(total_n_p1,sum(max_total_n_p1))


      # compute all CCR values and store the values for protocol 2
      results_n_p2 = collections.defaultdict(dict)
      max_total_n_p2 = numpy.zeros(len(fpr_thresholds))
      results_u_p2 = collections.defaultdict(dict)
      max_total_u_p2 = numpy.zeros(len(fpr_thresholds))

      ccrs_p2 = ccr_at_fpr(gt2[protocols2], scores2[protocols2][loss][algorithm], fpr_thresholds, unk_label=-1)
      results_n_p2[algorithm][loss] = ccrs_p2
      max_total_n_p2 = nonemax(max_total_n_p2, ccrs_p2)

      ccrs_p2 = ccr_at_fpr(gt2[protocols2], scores2[protocols2][loss][algorithm], fpr_thresholds, unk_label=-2)
      results_u_p2[algorithm][loss] = ccrs_p2
      max_total_u_p2 = nonemax(max_total_u_p2, ccrs_p2)

      total_u_p2 = max_total_u_p2
      total_u_p2 = np.append(total_u_p2,sum(max_total_u_p2))

      total_n_p2 = max_total_n_p2
      total_n_p2 = np.append(total_n_p2,sum(max_total_n_p2))


      # compute all CCR values and store the values for protocol 3
      results_n_p3 = collections.defaultdict(dict)
      max_total_n_p3 = numpy.zeros(len(fpr_thresholds))
      results_u_p3 = collections.defaultdict(dict)
      max_total_u_p3 = numpy.zeros(len(fpr_thresholds))

      ccrs_p3 = ccr_at_fpr(gt3[protocols3], scores3[protocols3][loss][algorithm], fpr_thresholds, unk_label=-1)
      results_n_p3[algorithm][loss] = ccrs_p3
      max_total_n_p3 = nonemax(max_total_n_p3, ccrs_p3)

      ccrs_p3 = ccr_at_fpr(gt3[protocols3], scores3[protocols3][loss][algorithm], fpr_thresholds, unk_label=-2)
      results_u_p3[algorithm][loss] = ccrs_p3
      max_total_u_p3 = nonemax(max_total_u_p3, ccrs_p3)

      total_u_p3 = max_total_u_p3
      total_u_p3 = np.append(total_u_p3,sum(max_total_u_p3))

      total_n_p3 = max_total_n_p3
      total_n_p3 = np.append(total_n_p3,sum(max_total_n_p3))

      #write latex table to the txt file
      with open(latex_file, "w") as f:
        f.write("\\begin{table}[H]\n")
        f.write("\\centering \n")
        f.write("\\begin{tabular}{c |c c c|c c c c | c} \n")
        f.write(f"\\textbf{{{title_column} }}& \multicolumn{{3}}{{c|}}{{\\textbf{{Negative}}}} & \multicolumn{{3}}{{c}}{{\\textbf{{Unknown}}}}& \\textbf{{Acc}} & \\textbf{{$\sum$}}\\\ \n")
        f.write("\\rule{0pt}{12pt} \n")
        f.write("\\textbf{ }& $10e-3$& $10e-2$& $10e-1$ & $10e-3$& $10e-2$& $10e-1$ & $1.$ & \\textbf{ } \\\ \n")
        f.write("\\rule{0pt}{12pt} \n")
        f.write(f"\\textit{{{title_1}}}  & {total_n_p1[0]:.3f} & {total_n_p1[1]:.3f} & {total_n_p1[2]:.3f} & {total_u_p1[0]:.3f}& {total_u_p1[1]:.3f} & {total_u_p1[2]:.3f} & {total_u_p1[3]:.3f}& {total_u_p1[4]:.3f}\\\ \n")
        f.write("\\rule{0pt}{8pt} \n")
        f.write(f"\\textit{{{title_2}}}  & {total_n_p2[0]:.3f} & {total_n_p2[1]:.3f} & {total_n_p2[2]:.3f} & {total_u_p2[0]:.3f}& {total_u_p2[1]:.3f} & {total_u_p2[2]:.3f} & {total_u_p2[3]:.3f}& {total_u_p2[4]:.3f}\\\ \n")
        f.write("\\rule{0pt}{8pt} \n")
        f.write(f"\\textit{{{title_3}}}  & {total_n_p3[0]:.3f} & {total_n_p3[1]:.3f} & {total_n_p3[2]:.3f} & {total_u_p3[0]:.3f}& {total_u_p3[1]:.3f} & {total_u_p3[2]:.3f} & {total_u_p3[3]:.3f}& {total_u_p3[4]:.3f}\\\ \n")
        f.write("\\end{tabular} \n")
        f.write(f"\\caption{{{caption}}}\n")
        f.write("\\end{table}")



In [ ]:
#parameters
use_best = False
protocols = (2)
algorithm = 'threshold'
protocol = 2
method = 'mixup'
loss = 'entropic'

#enter the data for the three protocols
scores, gt = load_scores(use_best, algorithm, loss,1 ,method,'test', 'p1')
scores1, gt1 = load_scores(use_best, algorithm, loss, 2 ,method,'test', 'p2')
scores2, gt2 = load_scores(use_best, algorithm, loss, 3 ,method,'test', 'p3')

#make the table
ccr_table_new(1,2,3,"Experiments","$P1$ ","$P2$","$P3$", algorithm, loss, scores, gt,scores1, gt1,scores2, gt2, 'Title', 'CCR@FPR for all protocols')
